# Inference from DS Workbench

In [ ]:
import sys
import os

PATH_TO_SRC = "/home/ksatola/work/src"
PATH_TO_DATA_FILE = "/home/ksatola/work/data/data_pipeline/training/data.csv"
sys.path.insert(1, PATH_TO_SRC)

import pandas as pd

from data_utils import get_train_test_split_for_stock

## Get data for inference

In [ ]:
X_train, X_test, y_train, y_test = get_train_test_split_for_stock(PATH_TO_DATA_FILE)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
X_test.head()

In [ ]:
X_test.iloc[:1]

## Import model from MLflow

In [ ]:
import mlflow
logged_model = '/data/artifacts/3/df2f6a41c8fd4c53809b9905a418b418/artifacts/model' # logistic reg

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

In [ ]:
loaded_model.predict(pd.DataFrame(X_test.iloc[:5]))

## Run model on local server

In [ ]:
# Get Run ID from MLmodel file (in MLflow UI or from the artifacts folder)
# https://www.mlflow.org/docs/latest/quickstart.html

# This option does not seem to work as mlflow does not show models COMMAND in the DS_workbench terminal... 
# It works from a host computer -> see notebook 290 run from the HOST

# Run in DS_workbench terminal (or locally from any Docker container) - NOT from the HOST
mlflow models serve -m runs:/df2f6a41c8fd4c53809b9905a418b418/model --port 1234
    
# Listens at: http://127.0.0.1:1234

In [ ]:
test_json = X_test.iloc[:5].to_json(orient='split')
test_json

In [ ]:
import requests

result = requests.post(url="http://127.0.0.1:1234/invocations",
                       data=test_json,
                       headers={'Content-Type':'application/json'})

In [ ]:
result.json()